In [1]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [4]:
import os

filepath = os.path.join(
    '/content/drive/MyDrive/My Notebooks/MY_2022 ML DS Master Class/01-Linear Regression/', 'Advertising.csv')


In [5]:
df = pd.read_csv(filepath)


In [6]:
X = df.drop('sales', axis=1)


In [7]:
y = df['sales']


In [36]:
df.head()


,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [37]:
df.shape


(200, 4)

### Polynomial Conversion


In [8]:
from sklearn.preprocessing import PolynomialFeatures


In [10]:
poly_converter = PolynomialFeatures(degree=3, include_bias=False)


In [11]:
poly_features = poly_converter.fit_transform(X)


In [12]:
poly_features.shape


(200, 19)

### Train | Test Split


In [13]:
from sklearn.model_selection import train_test_split


In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    poly_features, y, test_size=0.3, random_state=101)


In [16]:
X_train.shape


(140, 19)

---

---

## Scaling the Data

While our particular data set has all the values in the same order of magnitude ($1000s of dollars spent), typically that won't be the case on a dataset, and since the mathematics behind regularized models will sum coefficients together, its important to standardize the features. Review the theory videos for more info, as well as a discussion on why we only **fit** to the training data, and **transform** on both sets separately.


In [18]:
# scaling the data
from sklearn.preprocessing import StandardScaler


In [19]:
scaler = StandardScaler()


In [38]:
scaler.fit(X_train)


StandardScaler()

In [39]:
X_train = scaler.transform(X_train)


In [40]:
X_test = scaler.transform(X_test)


In [41]:
X_train[0]


array([ 0.49300171, -0.33994238,  1.61586707,  0.28407363, -0.02568776,
        1.49677566, -0.59023161,  0.41659155,  1.6137853 ,  0.08057172,
       -0.05392229,  1.01524393, -0.36986163,  0.52457967,  1.48737034,
       -0.66096022, -0.16360242,  0.54694754,  1.37075536])

In [34]:
# help(StandardScaler)


## Ridge Regression


In [42]:
from sklearn.linear_model import Ridge


In [44]:
# help(Ridge)


In [45]:
ridge_model = Ridge(alpha=10)


In [46]:
ridge_model.fit(X_train, y_train)


Ridge(alpha=10)

In [47]:
test_predictions = ridge_model.predict(X_test)


In [48]:
from sklearn.metrics import mean_absolute_error, mean_squared_error


In [49]:
MAE = mean_absolute_error(y_test, test_predictions)


In [51]:
RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))


In [52]:
print('MAE: ', MAE)
print('RMSE: ', RMSE)


MAE:  0.5774404204714164
RMSE:  0.8946386461319645


### Choosing an alpha value with Cross-Validation


In [53]:
from sklearn.linear_model import RidgeCV


In [61]:
ridge_cv_model = RidgeCV(alphas=(0.1, 1.0, 10),
                         scoring='neg_mean_absolute_error')


In [62]:
ridge_cv_model.fit(X_train, y_train)


RidgeCV(alphas=array([ 0.1,  1. , 10. ]), scoring='neg_mean_absolute_error')

In [63]:
ridge_cv_model.alpha_


0.1

In [64]:
# from sklearn.metrics import SCORERS

# SCORERS.keys()


dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

In [67]:
test_predictions = ridge_cv_model.predict(X_test)


In [70]:
MAE = mean_absolute_error(y_test, test_predictions)
RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))


In [71]:
print("\t\t\t New values after adding Ridge with Cross-Validation")
print('MAE: ', MAE)
print('RMSE: ', RMSE)


			 New values after adding Ridge with Cross-Validation
MAE:  0.4273774884337424
RMSE:  0.6180719926936866


In [72]:
ridge_cv_model.coef_


array([ 5.40769392,  0.5885865 ,  0.40390395, -6.18263924,  4.59607939,
       -1.18789654, -1.15200458,  0.57837796, -0.1261586 ,  2.5569777 ,
       -1.38900471,  0.86059434,  0.72219553, -0.26129256,  0.17870787,
        0.44353612, -0.21362436, -0.04622473, -0.06441449])

In [73]:
ridge_cv_model.best_score_


-0.37492233402929603

### LASSO - Least Absolute Shrinkage and Selection Operator


In [76]:
from sklearn.linear_model import LassoCV


In [107]:
lasso_cv_model = LassoCV(eps=0.001, n_alphas=100, max_iter=1000000)


In [108]:
lasso_cv_model.fit(X_train, y_train)


LassoCV(max_iter=1000000)

In [109]:
lasso_cv_model.alpha_


0.004943070909225837

In [110]:
test_predictions = lasso_cv_model.predict(X_test)


In [111]:
MAE = mean_absolute_error(y_test, test_predictions)
RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))


In [113]:
print("\t\t\t New values after adding LASSO with Cross-Validation")
print('MAE: ', MAE)
print('RMSE: ', RMSE)


			 New values after adding LASSO with Cross-Validation
MAE:  0.43350346185901
RMSE:  0.6063140748984103


In [112]:
lasso_cv_model.coef_


array([ 4.86023329,  0.12544598,  0.20746872, -4.99250395,  4.38026519,
       -0.22977201, -0.        ,  0.07267717, -0.        ,  1.77780246,
       -0.69614918, -0.        ,  0.12044132, -0.        , -0.        ,
       -0.        ,  0.        ,  0.        , -0.        ])

## ElasticNet


In [92]:
from sklearn.linear_model import ElasticNetCV


In [95]:
elastic_model = ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], eps=0.001, n_alphas=100, max_iter=10000000)


In [96]:
elastic_model.fit(X_train, y_train)


ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], max_iter=10000000)

In [114]:
elastic_model.l1_ratio_


1.0

In [115]:
elastic_model.alpha_


0.004943070909225837

In [116]:
lasso_cv_model.alpha_


0.004943070909225837

In [ ]:
test_predictions = elastic_model.predict(X_test)


In [ ]:
MAE = mean_absolute_error(y_test, test_predictions)
RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))


In [ ]:
print("\t\t\t New values after Elastic Model with Cross-Validation")
print('MAE: ', MAE)
print('RMSE: ', RMSE)
